In [ ]:
import numpy as np
import open3d as o3d
import threading

In [ ]:
def threaded_draw(*args, **kwargs):
    def workload():
        o3d.visualization.draw(*args, **kwargs)
    thread = threading.Thread(target=workload)
    thread.start()

In [ ]:
big_bbox = o3d.geometry.AxisAlignedBoundingBox((0, 0, 0),(30, 20, 10))
threaded_draw([big_bbox], title="Open3D", width=640, height=480)

In [ ]:
visualizer = o3d.WebVisualizer()
visualizer.show()